## 7.4 합성곱/풀링 계층 구현
---

In [4]:
########im2col 사용해보기 ##############
import numpy as np
import sys, os
sys.path.append(os.pardir)
from util import im2col


x1 = np.random.rand(1, 3, 7, 7) #데이터 1, 채널 3, 높이 7, 너비 7
col1 = im2col(x1, 5, 5, stride=1, pad=0) # (input_data, filter_h, filter_w, stribe, pad)
print(col1.shape) #im2col의 결과는 2차원으로 출력 -> 4차원으로 바꿔줘야함

x2 = np.random.rand(10, 3, 7, 7) #데이터가 x1의 10배
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [5]:
####################im2col 사용해서 합성곱계층 구현#######################
class Convolution:
	def __init__(self, W, b, stride=1, pad=0):
		self.W = W #필터(가중치)
		self.b = b #편향
		self.stride = stride
		self.pad = pad

	def forward(self, x):
		FN, C, FH, FW = self.W.shape #FN:필터갯수, 
		N, C, H, W = x.shape
		out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
		out_w = int(1 + (W + 2/self.pad - FW) / self.stride)

########중요! 입력데이터 -> im2col로 전개 -> 필터로 reshape해서 2차원으로 전개############
		col = im2col(x, FH, FW, self.stride, self.pad)
		col_W = self.W.reshape(FN, -1).T
		out = np.dot(col, col_W) + self.b

		out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

		return out

In [7]:
#################풀링 레이어 구현하기#########################
class Pooling:
	def __init__(self, pool_h, pool_w, stride=1, pad=0):
		self.pool_h = pool_h
		self.pool_w = pool_w
		self.stride = stride
		self.pad = pad

	def forward(self, x):
		N, C, H, W = x.shape
		out_h = int(1 + (H + self.pool_h) / self.stride)
		out_w = int(1 + (W + self.pool_w) / self.stride)

		col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
		col = col.reshape(-1, self.pool_h*self.pool_w)

		out = np.max(col, axis=1)

		out = out.reshape(N, out_h, out_w, C).tramspose(0, 3, 1, 2)

		return out

### CNN code 구현
----


In [13]:
###############CNN 만들어보기#################
class SimpleConvNet:

###############CNN 함수 초기화#################
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num': 30, 'filter_size': 5,
                             'pad': 0, 'stride': 1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):

        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) *
                               (conv_output_size/2))
###############가중치 매개변수 초기화#######################
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

#################CNN 구현계층#########################
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
#####################초기화 끝##########################

####################추론 및 loss 계산##########################
    def predict(self, x):
        """추론을 수행"""
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        """손실함수 값 계산"""
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

################역전파로 기울기 구하기######################
    def gradient(self, x, t):
        """오차역전파법으로 기울기를 구함"""
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads